In [1]:
from deep_translator import GoogleTranslator
import deepl
from data_class import Data
import time
import random

from configuration.config import API_KEY_DEEPL

In [2]:
data_manager = Data()

In [10]:
# GoogleTranslator

def translate_column_to_english(data_manager: Data, table_name: str = 'train',
                              source_column: str = 'task', 
                              target_column: str = 'translated_google'):
    data_manager.prepare_translation_column(table_name, target_column)
    
    rows = data_manager.get_data_for_translation(table_name, source_column)
    
    print(f"Начинаем перевод {len(rows)} записей...")
    
    for i, (row_id, text) in enumerate(rows, 1):
        if text:
            translated_text = translate_text(text, 'en')
            
            data_manager.save_translated_text(table_name, target_column, 
                                            row_id, translated_text)
            
    
            if i % 5 == 0:  
                time.sleep(random.uniform(0.5, 1.5))
            
            if i % 100 == 0: 
                data_manager.commit_changes()
                print(f"Переведено {i}/{len(rows)} записей...")
    
    data_manager.commit_changes()
    print(f"Перевод завершен! Всего переведено {len(rows)} записей.")


def translate_text(text: str, target_language: str = 'en') -> str:
    """Переводит текст на указанный язык с несколькими попытками и задержками"""
    if not text or not isinstance(text, str):
        return text
    
    max_attempts = 3
    
    for attempt in range(max_attempts):
        try:
            translator = GoogleTranslator(source='auto', target=target_language)
            translated = translator.translate(text)
            return translated
            
        except Exception as e:
            print(f"Попытка {attempt + 1}/{max_attempts} не удалась для '{text[:50]}...': {e}")
            
            if attempt < max_attempts - 1:
                delay = random.uniform(1, 5)
                print(f"Ждем {delay:.1f} секунд перед следующей попыткой...")
                time.sleep(delay)
            else:
                print(f"Все попытки исчерпаны для текста: '{text[:50]}...'")
    
    return text
    

translate_column_to_english(data_manager)
translate_column_to_english(data_manager, table_name='test')

Начинаем перевод 1000 записей...
Переведено 100/1000 записей...
Переведено 200/1000 записей...
Переведено 300/1000 записей...
Переведено 400/1000 записей...
Переведено 500/1000 записей...
Переведено 600/1000 записей...
Переведено 700/1000 записей...
Переведено 800/1000 записей...
Переведено 900/1000 записей...
Переведено 1000/1000 записей...
Перевод завершен! Всего переведено 1000 записей.
Начинаем перевод 200 записей...
Переведено 100/200 записей...
Переведено 200/200 записей...
Перевод завершен! Всего переведено 200 записей.


In [3]:
# DEEPL
def translate_text_deepl(text: str, api_key: str, target_language: str = 'EN-US') -> str:
    if not text or not isinstance(text, str):
        return text
    
    max_attempts = 3
    
    for attempt in range(max_attempts):
        try:
            translator = deepl.Translator(api_key)
            result = translator.translate_text(text, target_lang=target_language)
            return str(result)
            
        except deepl.exceptions.AuthorizationException:
            print(f"Ошибка авторизации DeepL API. Проверьте ключ API.")
            return text
            
        except deepl.exceptions.QuotaExceededException:
            print(f"Превышена квота DeepL API.")
            return text
            
        except Exception as e:
            print(e)
            print(f"Попытка {attempt + 1}/{max_attempts} не удалась для DeepL '{text[:50]}...': {e}")
            
            if attempt < max_attempts - 1:
                delay = random.uniform(1, 3)
                print(f"Ждем {delay:.1f} секунд перед следующей попыткой...")
                time.sleep(delay)
            else:
                print(f"Все попытки DeepL исчерпаны для текста: '{text[:50]}...'")
    
    return text


def translate_column_to_english_deepl(data_manager: Data, api_key: str,
                                    table_name: str = 'train',
                                    source_column: str = 'task', 
                                    target_column: str = 'translated_deepl'):

    data_manager.prepare_translation_column(table_name, target_column)
    

    rows = data_manager.get_data_for_translation_filtered(table_name, source_column, target_column)
    
    print(f"Начинаем перевод DeepL {len(rows)} записей...")
    
    for i, (row_id, text) in enumerate(rows, 1):
        if text:
            translated_text = translate_text_deepl(text, api_key, 'EN-US')
            
            data_manager.save_translated_text(table_name, target_column, 
                                            row_id, translated_text)
            
            if i % 5 == 0:
                time.sleep(random.uniform(0.3, 1.0))
            
            if i % 100 == 0:  
                data_manager.commit_changes()
                print(f"Переведено DeepL {i}/{len(rows)} записей...")
    
    data_manager.commit_changes()
    print(f"Перевод DeepL завершен! Всего переведено {len(rows)} записей.")


translate_column_to_english_deepl(data_manager, API_KEY_DEEPL)
translate_column_to_english_deepl(data_manager, API_KEY_DEEPL, table_name='test')
    

Начинаем перевод DeepL 1000 записей...
Переведено DeepL 100/1000 записей...
Переведено DeepL 200/1000 записей...
Переведено DeepL 300/1000 записей...
Connection failed: HTTPSConnectionPool(host='api-free.deepl.com', port=443): Max retries exceeded with url: /v2/translate (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001E2B70039B0>: Failed to resolve 'api-free.deepl.com' ([Errno 11001] getaddrinfo failed)"))
Попытка 1/3 не удалась для DeepL 'How many times does the digit 8 appear in the list...': Connection failed: HTTPSConnectionPool(host='api-free.deepl.com', port=443): Max retries exceeded with url: /v2/translate (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001E2B70039B0>: Failed to resolve 'api-free.deepl.com' ([Errno 11001] getaddrinfo failed)"))
Ждем 2.1 секунд перед следующей попыткой...
Переведено DeepL 400/1000 записей...
Переведено DeepL 500/1000 записей...
Переведено DeepL 600/1000 записей...
Переведен